In [2]:
!pip install openai langchain

In [3]:
! pip install --upgrade --quiet langchain-nvidia-ai-endpoints
! pip install langchain-nvidia-ai-endpoints
! pip install PyPDF2

In [5]:
# from openai import OpenAI
# from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import getpass
import os

In [ ]:
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key


In [29]:
#client = OpenAI(
#  base_url = "https://integrate.api.nvidia.com/v1",
#  api_key = "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"
#)

C:\Users\hcchou\AppData\Local\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
## https://python.langchain.com/v0.2/docs/integrations/chat/nvidia_ai_endpoints/
from langchain_nvidia_ai_endpoints import ChatNVIDIA

In [6]:
# testing
llm = ChatNVIDIA(model="meta/llama2-70b", max_tokens=500) 
result = llm.invoke("What is the weather in Taiwan like in June?")
print(result.content)

  In June, Taiwan'лле bunch of rain. It's like a wholeseason of mizzle. You'll want to pack an umbrella, or maybe even a snorkel.
The average temperature in June is around 25 degrees Celsius (77 degrees Fahrenheit), with high humidity. It can feel pretty hot and sticky, especially when it's not raining.
But don't worry, the rain isn't necessarily a bad thing. It's a great time to see Taiwan's beautiful waterfalls and lush green landscapes in all their glory. Just be prepared for a wet and wild adventure!


In [73]:
## evaluate essay with text only
def evaluate_essay(essay_text):
    # Set up the language model
#    llm = OpenAI(temperature=0.1)

    # Define a prompt template for the essay evaluation
    prompt = PromptTemplate(
        input_variables=["essay"],
        template=""" Please evaluate the quality of the following essay:

        {essay}

        Provide a detailed analysis of the essay's strengths and weaknesses, including an overall quality score on a scale of 1 to 10.
        """
    )

    # Generate the essay evaluation using the language model
    result = llm(prompt.format(essay=essay_text))

    return result

In [37]:
##long text for essay, SyntaxError: unterminated string literal  use file link instead
## essay_text = "This is the text of the essay you want to evaluate."
## evaluation = evaluate_essay(essay_text)
## print(evaluation)

C:\Users\hcchou\AppData\Local\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


ValueError: Invalid message: ' ' of type <class 'str'>

In [ ]:
## use file path of the essay

import os
import PyPDF2
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_nvidia_ai_endpoints.chat_models import ChatNVIDIA
from langchain.schema import AIMessage

In [ ]:


def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        essay_text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            essay_text += page.extract_text()
    return essay_text

def evaluate_essay_quality(pdf_file_path):
    # Extract text from PDF file
    essay_text = extract_text_from_pdf(pdf_file_path)

    # Set up the language model
    llm = ChatNVIDIA(model="meta/llama2-70b", max_tokens=1000, temperature=0)

    # Define a prompt template for the essay evaluation
    prompt = PromptTemplate(
        input_variables=["essay"],
        template="""
        Please evaluate the quality of the following essay:

        {essay}

        Provide a detailed analysis of the strengths and weaknesses of the essay, and give an overall quality score on a scale of 1 to 10.
        """
    )

    # Generate the essay evaluation using the language model
    result = llm([AIMessage(content=prompt.format(essay=essay_text))])

    return result.content



In [ ]:

# Example usage
pdf_file_path = '/Users/hchou/Downloads/Essays and Evaluations/#1/Essay#1.pdf'
evaluation = evaluate_essay_quality(pdf_file_path)
print(evaluation)

In [6]:
## evaluated by a stricted technical background teacher

def evaluate_essay_quality_s(pdf_file_path):
    # Extract text from PDF file
    essay_text = extract_text_from_pdf(pdf_file_path)

    # Set up the language model
    llm = ChatNVIDIA(model="meta/llama2-70b", max_tokens=1000, temperature=0)

    # Define a prompt template for the essay evaluation
    prompt = PromptTemplate(
        input_variables=["essay"],
        template="""
        As a strict technical background teacher, I will provide a thorough evaluation of the following essay:

    {essay}

    The essay will be assessed on the following criteria:
    - Clarity and conciseness of the writing
    - Logical flow and organization of the content
    - Appropriate use of technical terms and concepts
    - Depth of understanding and analysis of the subject matter
    - Overall effectiveness in conveying the key points

    Please note that I will be evaluating this essay with a critical eye and a focus on technical excellence, and give an overall quality score on a scale of 1 to 10.
        """
    )

    # Generate the essay evaluation using the language model
    result_s = llm([AIMessage(content=prompt.format(essay=essay_text))])

    return result_s.content

In [13]:
## evaluate another essay
pdf_file_path = '/Users/hchou/Downloads/Essays and Evaluations/#1/Essay#2.pdf"
evaluation = evaluate_essay_quality(pdf_file_path)
print(evaluation)

SyntaxError: EOL while scanning string literal (428183186.py, line 2)

In [7]:
# essay2 evaluated by a strict technology teacher
pdf_file_path = 'C:/Users/hcchou/Downloads/Nvidia contest/Essay#2.pdf'
evaluation = evaluate_essay_quality_s(pdf_file_path)
print(evaluation)

C:\Users\hcchou\AppData\Local\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


<s> The essay is well-structured and easy to follow, with a clear introduction, body, and conclusion. The writer effectively uses transitions to connect their ideas and provides a clear thesis statement that guides the reader through the essay. The essay also demonstrates a good understanding of the topic, with the writer effectively using technical terms and concepts to support their arguments.

However, there are some areas where the essay could be improved. Firstly, the introduction could be stronger. While the writer does a good job of introducing the topic and providing some background information, they could do more to engage the reader and provide a clearer sense of the essay's focus. Additionally, some of the sentences could be shorter and more concise, making the essay easier to read.

Furthermore, while the writer effectively uses technical terms and concepts, there are some instances where they could be more precise in their language. For example, the writer uses the term "t

In [19]:
## debate take 2
## debate file in pdf

## try
def evaluate_essay_quality(pdf_file_path):
    # Extract text from PDF file
    essay_text = extract_text_from_pdf(pdf_file_path)

    # Set up the language model
    llm = ChatNVIDIA(model="meta/llama2-70b", max_tokens=1000, temperature=0)

    # Define a prompt template for the essay evaluation
    prompt = PromptTemplate(
        input_variables=["essay"],
        template="""
        Please evaluate the quality of the following essay:

        {essay}

        Provide a detailed analysis of the strengths and weaknesses of the essay, and give an overall quality score on a scale of 1 to 10.
        """
    )

    # Generate the essay evaluation using the language model
    result = llm([AIMessage(content=prompt.format(essay=essay_text))])

    return result.content

def debate_with_llm(pdf_ev_file_path, human_opinion):
    # Extract text from PDF file
    llm_response = extract_text_from_pdf(pdf_ev_file_path)
    
    # Set up the language model
    llm = ChatNVIDIA(model="meta/llama2-70b", max_tokens=1000, temperature=0)

    # Define prompt template
    prompt = PromptTemplate(
        input_variables=["llm_evaluation", "human_opinion"],
        template="""
        The language model has provided the following evaluation:

        {llm_evaluation}

        As a human teacher has given opinion on the essay:

        {human_opinion}

        The human teacher would like you to revise your evaluation. Please add human teacher's opinion and summary the evaluation in 200 words. Revise and give an overall quality score on a scale of 1 to 10.
        """
    )
    human_response = llm([AIMessage(content=prompt.format(llm_evaluation=llm_response, human_opinion=human_opinion))])
    return human_response.content

In [20]:
pdf_ev_file_path = '/Users/hchou/Downloads/Essays and Evaluations/#1/Essay#1_llm_ev.pdf'

human_opinion = """
As a human teacher, I agree with your opinion. Yet I believe the author demonstrates a good understanding of the topic and the potential of extending the result further. 

Overall, I would rate the essay a 8 out of 10.
"""

human_response = debate_with_llm(pdf_ev_file_path, human_opinion)

print(human_response)


<s> The revised evaluation should include the following:

1. A clear thesis statement that summarizes the main point of the essay.
2. More concise sentences that are easier to understand.
3. More specific examples of the author's contributions to the project, such as detailed descriptions of their designs and how they addressed the problem of the train gap.
4. More detailed feedback and analysis of the author's designs and ideas.
5. A more critical evaluation of the project's successes and limitations.

Here is a revised evaluation of the essay:

The essay provides a detailed description of the author's experience in a class project, where they designed mechanisms to fill the gap between a train and a station platform. The author reflects on what they learned from the class, including collaboration, communication, and critical thinking skills, and how they applied these skills in the project. The essay is well-organized and easy to follow, with a clear introduction, body, and conclusio

In [ ]:
# coversation chain : discussion take 4

In [89]:
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [8]:
## compare two essays, tried manytimes, it seems the model can not take two pdf inputs.

def compare_essay_quality(pdf_file_path1, pdf_file_path2):
    # Extract text from PDF file 1
    with open(pdf_file_path1, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        essay_text1 = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            essay_text1 += page.extract_text()

    # Extract text from PDF file 2
    with open(pdf_file_path2, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        essay_text2 = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            essay_text2 += page.extract_text()

    # Set up the language model
    llm = ChatNVIDIA(model="meta/llama2-70b", max_tokens=2000, temperature=0)

    # Define a prompt template for the comparative essay evaluation
    prompt = PromptTemplate(
        input_variables=["essay1", "essay2"],
        template="""
        Please analysis which essay has better quality.

        Essay 1:
        {essay1}

        Please evaluate the quality of the following essay:

        Essay 2:
        {essay2}

        # Provide the strengths and weaknesses of each essay with under 200 words each, and give an overall quality score for each essay on a scale of 1 to 10. Conclude with a recommendation on which essay has the higher quality.
        """
    )

    # Generate the comparative essay evaluation using the language model
    result_c = llm(prompt.format(essay1=essay_text1, essay2=essay_text2))

    return result_c


In [9]:
pdf_file_path1 = 'C:/Users/hcchou/Downloads/Nvidia contest/Essay#1.pdf'
pdf_file_path2 = 'C:/Users/hcchou/Downloads/Nvidia contest/Essay#2.pdf'
comparison_c = compare_essay_quality(pdf_file_path1, pdf_file_path2)
print(comparison)

ValueError: Invalid message: '\n' of type <class 'str'>